<a href="https://colab.research.google.com/github/TroTusk/Contacts-classification/blob/main/contacts_classifcation_structural_bioinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGB install and imports
Sets up XGBoost, mounts Drive, and imports all libraries (NumPy/Pandas, scikit-learn, XGBoost).

In [1]:
!pip -q install xgboost

from google.colab import drive
drive.mount('/content/drive')

import os, glob, time, json, joblib, numpy as np, pandas as pd
from pathlib import Path

from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, matthews_corrcoef,
    classification_report, confusion_matrix, average_precision_score, roc_auc_score
)
from xgboost import XGBClassifier


Mounted at /content/drive


# Paths, options, and helper functions
Defines project paths and small helpers:


*   load_from_features_ring to read multiple TSVs
*   ap_auc_macro to compute macro AP/AUC
*   subsample_binary to build balanced one-vs-rest subsets quickly.










## Prerequisites (read before running)

1) Download or clone the project locally, then **place the whole project folder in your Google Drive** at:

/content/drive/MyDrive/progetto_bioinfo_finale/project

If you prefer a different location, you must update `DRIVE_PROJECT_DIR` in the next cell.


In [2]:
# Project root on Drive
DRIVE_PROJECT_DIR = "/content/drive/MyDrive/progetto_bioinfo_finale/project" # edit if you prefer a different location
FEATURES_DIR = os.path.join(DRIVE_PROJECT_DIR, "features_ring")
FALLBACK_DATASET = os.path.join(DRIVE_PROJECT_DIR, "complete_training_dataset.tsv")

# output names
ts = time.strftime("%Y%m%d_%H%M%S")
MODEL_PATH = os.path.join(DRIVE_PROJECT_DIR, f"contact_models_{ts}.joblib")

# options
MAX_FILES   = 1000  # reduce for lighter training
RANDOM_SEED = 42

# print directories
print("Project:", DRIVE_PROJECT_DIR)
print("features_ring exists:", os.path.isdir(FEATURES_DIR))
print("fallback dataset exists:", os.path.exists(FALLBACK_DATASET))

# set rng
rng = np.random.default_rng(RANDOM_SEED)


"""
load up to `max_files` TSVs from `root`, add a `__pdb__` column from the filename,
concatenate them, and return a single DataFrame (or None if nothing loads).
"""
def load_from_features_ring(root, max_files=MAX_FILES):

    files = sorted(glob.glob(os.path.join(root, "*.tsv")))
    if not files:
        return None
    if max_files and len(files) > max_files:
        files = list(rng.choice(files, size=max_files, replace=False))
    frames = []
    for fp in files:
        try:
            df = pd.read_csv(fp, sep="\t")
            df["__pdb__"] = Path(fp).stem
            frames.append(df)
        except Exception:
            pass
    if not frames:
        return None
    return pd.concat(frames, ignore_index=True)

# compute macro Average Precision and macro ROC AUC in one-vs-rest. returns (ap_macro, auc_macro)
def ap_auc_macro(y_true: pd.Series, Proba: pd.DataFrame, classes: list):
    Yb = pd.get_dummies(pd.Categorical(y_true, categories=classes)).reindex(columns=classes, fill_value=0).to_numpy()
    A  = Proba.loc[y_true.index, classes].to_numpy()
    aps, aucs = [], []
    for i in range(len(classes)):
        pos = Yb[:, i].sum()
        neg = len(Yb) - pos
        if pos > 0:
            aps.append(average_precision_score(Yb[:, i], A[:, i]))
        if pos > 0 and neg > 0:
            aucs.append(roc_auc_score(Yb[:, i], A[:, i]))
    apm  = float(np.mean(aps))  if aps  else np.nan
    aucm = float(np.mean(aucs)) if aucs else np.nan
    return apm, aucm

# build a one-vs-rest training subset
def subsample_binary(y_series, pos_label, neg_pos_ratio=3, max_pos=None, rnd=rng):
    idx_pos = np.where(y_series.values == pos_label)[0]
    idx_neg = np.where(y_series.values != pos_label)[0]
    if max_pos is not None and len(idx_pos) > max_pos:
        idx_pos = rnd.choice(idx_pos, size=max_pos, replace=False)
    n_neg = min(len(idx_neg), int(len(idx_pos) * neg_pos_ratio))
    if n_neg < len(idx_neg):
        idx_neg = rnd.choice(idx_neg, size=n_neg, replace=False)
    idx = np.concatenate([idx_pos, idx_neg])
    rnd.shuffle(idx)
    return idx


Project: /content/drive/MyDrive/progetto_bioinfo_finale/project
features_ring exists: True
fallback dataset exists: True


# Load dataset (features_ring first, fallback otherwise)
Loads per-PDB TSVs from features_ring/ or falls back to the merged TSV. Drops unlabeled rows and “Unclassified”. Prints basic dataset stats.

In [3]:
# try features_ring
df_all = load_from_features_ring(FEATURES_DIR, max_files=MAX_FILES)

In [4]:
# otherwise use the fallback dataset (training_dataset_completo.tsv present on the project folder)
if df_all is None and os.path.exists(FALLBACK_DATASET):
    df_all = pd.read_csv(FALLBACK_DATASET, sep="\t")
    print("using fallback dataset")
    if "__pdb__" not in df_all.columns and "pdb_id" in df_all.columns:
        df_all["__pdb__"] = df_all["pdb_id"].astype(str)

In [5]:

assert df_all is not None, "No data found. Ensure features_ring/ or training_dataset_completo.tsv is available."

# remove 'Unclassified' rows
df_all = df_all.dropna(subset=["Interaction"]).copy()
df_all = df_all[df_all["Interaction"].astype(str) != "Unclassified"].copy()

print("Rows:", len(df_all))
print("Columns:", len(df_all.columns))
print("Classes:", sorted(df_all["Interaction"].astype(str).unique().tolist()))


Rows: 479608
Columns: 33
Classes: ['HBOND', 'IONIC', 'PICATION', 'PIHBOND', 'PIPISTACK', 'SSBOND', 'VDW']


# Build features
Creates the feature matrix: numeric columns with mean imputation and one-hot encoding for categorical variables (optionally adds 3Di letters if present).

In [6]:
# numeric columns (spec)
num_cols = [
    "s_rsa","s_phi","s_psi","s_a1","s_a2","s_a3","s_a4","s_a5",
    "t_rsa","t_phi","t_psi","t_a1","t_a2","t_a3","t_a4","t_a5",
]
# keep only the numeric columns that actually exist in this dataset
num_cols = [c for c in num_cols if c in df_all.columns]

# categorical feature columns
cat_cols = ["s_resn","t_resn","s_ss8","t_ss8"]

# include 3Di letters if present in the files
for c in ["s_3di_letter","t_3di_letter"]:
    if c in df_all.columns:
        cat_cols.append(c)

# Build feature matrix
X_base = df_all[num_cols + cat_cols].copy()

# numeric imputation, cast to numeric and fill any missing values with the column mean
for c in num_cols:
    X_base[c] = pd.to_numeric(X_base[c]).fillna(X_base[c].mean())


# One-hot encode only the categorical columns that are present
present_cats = [c for c in cat_cols if c in X_base.columns]
X = pd.get_dummies(X_base, columns=present_cats, dummy_na=False)


# Targets (interaction labels) and grouping key (per-PDB) for group-wise splitting
y = df_all["Interaction"].astype(str).copy()
groups = df_all["__pdb__"].astype(str) if "__pdb__" in df_all.columns else pd.Series(["_all_"]*len(df_all))

print("Final #features:", X.shape[1])

Final #features: 114


# Split by PDB
Performs a train/validation split using PDB IDs so the same structure doesn’t leak across splits. Prints class distribution.

In [7]:

# splitting training and validation
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=RANDOM_SEED)
tr_idx, va_idx = next(gss.split(X, y, groups=groups))

# slice features and labels
X_tr, X_va = X.iloc[tr_idx].copy(), X.iloc[va_idx].copy()
y_tr, y_va = y.iloc[tr_idx].copy(), y.iloc[va_idx].copy()

# class list and the exact feature set used by the model
classes = sorted(y.unique().tolist())
model_columns = X_tr.columns.tolist()

print("Train rows:", len(X_tr), "Validation rows:", len(X_va))
print("Classes:", classes)
print("\nTraining distribution:")
print(y_tr.value_counts())


Train rows: 386784 Validation rows: 92824
Classes: ['HBOND', 'IONIC', 'PICATION', 'PIHBOND', 'PIPISTACK', 'SSBOND', 'VDW']

Training distribution:
Interaction
HBOND        218523
VDW          150475
PIPISTACK      7723
IONIC          7456
PICATION       1779
SSBOND          467
PIHBOND         361
Name: count, dtype: int64


# Train OVR with ExtraTrees (baseline)
Trains one ExtraTrees binary classifier per class on subsampled data (balanced positives/negatives).

In [8]:
# train one binary model per class (one-vs-rest) with light subsampling
models = {}
for i, c in enumerate(classes, 1):
    # pick all positives for class c and a limited number of negatives
    idx = subsample_binary(y_tr, c, neg_pos_ratio=3, max_pos=20000)
    Xt, yt = X_tr.iloc[idx], (y_tr.iloc[idx] == c).astype(int)

    # extra-trees
    clf = ExtraTreesClassifier(
        n_estimators=250,          # number of trees
        max_depth=None,
        min_samples_leaf=3,        # small leaves to reduce overfitting
        max_features="sqrt",
        class_weight="balanced_subsample",
        n_jobs=-1,                 # use all cores
        random_state=RANDOM_SEED,
    )

    # fit the binary classifier for class c
    clf.fit(Xt, yt)

    # store the model for this class
    models[c] = clf

    # progress print
    print(f"[ET  {i}/{len(classes)}] trained {c:9s} on {len(idx)} rows ({int(yt.sum())} positives)")


[ET  1/7] trained HBOND     on 80000 rows (20000 positives)
[ET  2/7] trained IONIC     on 29824 rows (7456 positives)
[ET  3/7] trained PICATION  on 7116 rows (1779 positives)
[ET  4/7] trained PIHBOND   on 1444 rows (361 positives)
[ET  5/7] trained PIPISTACK on 30892 rows (7723 positives)
[ET  6/7] trained SSBOND    on 1868 rows (467 positives)
[ET  7/7] trained VDW       on 80000 rows (20000 positives)


In [9]:
# Saving the model (opzional) uncomment the next rows to allow saving
# joblib.dump({"models": models, "columns": model_columns, "classes": classes}, MODEL_PATH)
# print("Saved:", MODEL_PATH)

# Validate ExtraTrees
Computes probabilities, predicts by argmax, and prints Accuracy, Balanced Accuracy, MCC, macro AP and AUC, plus a classification report and confusion matrix.

In [10]:
P_va_et = np.column_stack([models[c].predict_proba(X_va)[:, 1] for c in classes]).astype(np.float32)
P_va_et = pd.DataFrame(P_va_et, columns=classes, index=X_va.index)
y_pred_et = P_va_et.idxmax(axis=1)
# calculates all the metrics
acc  = accuracy_score(y_va, y_pred_et)
ba   = balanced_accuracy_score(y_va, y_pred_et)
mcc  = matthews_corrcoef(y_va, y_pred_et)
apm, aucm = ap_auc_macro(y_va, P_va_et, classes)

print("Validation (ExtraTrees, argmax)")
print("Accuracy:", f"{acc:.4f}")
print("Balanced Accuracy:", f"{ba:.4f}")
print("MCC:", f"{mcc:.4f}")
print("Average Precision (macro):", "n/a" if np.isnan(apm) else f"{apm:.4f}")
print("ROC AUC (macro):", "n/a" if np.isnan(aucm) else f"{aucm:.4f}")

print("\nClassification report:")
print(classification_report(y_va, y_pred_et, zero_division=0))

print("Confusion matrix:")
print(confusion_matrix(y_va, y_pred_et))


Validation (ExtraTrees, argmax)
Accuracy: 0.5001
Balanced Accuracy: 0.7618
MCC: 0.1915
Average Precision (macro): 0.3789
ROC AUC (macro): 0.8707

Classification report:
              precision    recall  f1-score   support

       HBOND       0.66      0.54      0.59     52476
       IONIC       0.18      0.99      0.31      1756
    PICATION       0.13      0.98      0.22       419
     PIHBOND       0.01      0.43      0.02        92
   PIPISTACK       0.46      1.00      0.63      1802
      SSBOND       0.41      1.00      0.59        98
         VDW       0.49      0.39      0.44     36181

    accuracy                           0.50     92824
   macro avg       0.33      0.76      0.40     92824
weighted avg       0.57      0.50      0.52     92824

Confusion matrix:
[[28081  4978  1463  2059   931    47 14917]
 [    2  1747     0     0     0     0     7]
 [    0     0   409     8     0     0     2]
 [    7     0    17    40    19     0     9]
 [    0     0     0     3  1798     

# Train OVR with XGBoost
Trains one XGBoost binary classifier per class with class imbalance handled via scale_pos_weight. Uses the same subsampling recipe. Saving is left commented.

In [11]:
# train one binary XGBoost model per class (one-vs-rest) with light subsampling
xgb_models = {}
for i, c in enumerate(classes, 1):
    # pick all positives for class c and a limited number of negatives
    idx = subsample_binary(y_tr, c, neg_pos_ratio=3, max_pos=20000)
    Xt, yt = X_tr.iloc[idx], (y_tr.iloc[idx] == c).astype(int)

    # compute class imbalance for this subset used by scale_pos_weight
    pos = int(yt.sum()); neg = len(yt) - pos
    spw = (neg / pos) if pos > 0 else 1.0

    # xgboost setup to generalizes well
    clf = XGBClassifier(
        n_estimators=400,          # number of boosting rounds
        max_depth=6,               # tree depth (bias/variance knob)
        learning_rate=0.10,        # step size per round
        subsample=0.85,            # row sampling for regularization
        colsample_bytree=0.70,     # feature sampling per tree
        reg_lambda=1.0,            # L2 regularization
        objective="binary:logistic",
        scale_pos_weight=spw,      # fix class imbalance within this subset
        n_jobs=-1,
        random_state=RANDOM_SEED,
    )

    # fit the binary classifier for class c
    clf.fit(Xt, yt)

    # store the model for this class
    xgb_models[c] = clf

    # progress log
    print(f"[XGB {i}/{len(classes)}] {c:9s} — {len(idx)} rows ({pos} pos)")


# Saving the model (opzional) uncomment the next rows to allow saving
# XGB_MODEL_PATH = os.path.join(DRIVE_PROJECT_DIR, f"xgb_models_{ts}.joblib")
# joblib.dump({"models": xgb_models, "columns": model_columns, "classes": classes}, XGB_MODEL_PATH)
# print("Saved:", XGB_MODEL_PATH)


[XGB 1/7] HBOND     — 80000 rows (20000 pos)
[XGB 2/7] IONIC     — 29824 rows (7456 pos)
[XGB 3/7] PICATION  — 7116 rows (1779 pos)
[XGB 4/7] PIHBOND   — 1444 rows (361 pos)
[XGB 5/7] PIPISTACK — 30892 rows (7723 pos)
[XGB 6/7] SSBOND    — 1868 rows (467 pos)
[XGB 7/7] VDW       — 80000 rows (20000 pos)


# Validate XGBoost
Same evaluation as cell 7 but for the XGBoost ensemble, so you can compare ExtraTrees vs XGBoost on the same validation split.

In [12]:
P_va_xgb = np.column_stack([xgb_models[c].predict_proba(X_va)[:,1] for c in classes]).astype(np.float32)
P_va_xgb = pd.DataFrame(P_va_xgb, columns=classes, index=X_va.index)
y_pred_xgb = P_va_xgb.idxmax(axis=1)
# calculates the metrics
acc  = accuracy_score(y_va, y_pred_xgb)
ba   = balanced_accuracy_score(y_va, y_pred_xgb)
mcc  = matthews_corrcoef(y_va, y_pred_xgb)
apm, aucm = ap_auc_macro(y_va, P_va_xgb, classes)

print("Validation (XGBoost, argmax)")
print("Accuracy:", f"{acc:.4f}")
print("Balanced Accuracy:", f"{ba:.4f}")
print("MCC:", f"{mcc:.4f}")
print("Average Precision (macro):", "n/a" if np.isnan(apm) else f"{apm:.4f}")
print("ROC AUC (macro):", "n/a" if np.isnan(aucm) else f"{aucm:.4f}")

print("\nClassification report:")
print(classification_report(y_va, y_pred_xgb, zero_division=0))

print("Confusion matrix:")
print(confusion_matrix(y_va, y_pred_xgb))


Validation (XGBoost, argmax)
Accuracy: 0.5121
Balanced Accuracy: 0.7321
MCC: 0.2025
Average Precision (macro): 0.3630
ROC AUC (macro): 0.8708

Classification report:
              precision    recall  f1-score   support

       HBOND       0.66      0.55      0.60     52476
       IONIC       0.19      0.99      0.32      1756
    PICATION       0.13      0.88      0.22       419
     PIHBOND       0.01      0.38      0.02        92
   PIPISTACK       0.45      0.91      0.60      1802
      SSBOND       0.41      1.00      0.58        98
         VDW       0.50      0.41      0.45     36181

    accuracy                           0.51     92824
   macro avg       0.34      0.73      0.40     92824
weighted avg       0.58      0.51      0.53     92824

Confusion matrix:
[[28642  4656  1338  1793   865    47 15135]
 [   10  1742     0     0     0     0     4]
 [    1     0   370    45     0     0     3]
 [   10     0    16    35    17     0    14]
 [    0     0     0   164  1637     0  

# Test XGB on a random PDB
Builds features for a single PDB TSV, runs the XGBoost ensemble, and reports the metrics. Shows a small table of mismatches.

In [13]:
# test on a random TSV from features_ring
import os, glob, numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, matthews_corrcoef,
    classification_report, confusion_matrix, average_precision_score, roc_auc_score
)

# check if the trained model exists
if len(xgb_models) > 0:
    ensemble = xgb_models
    model_name = "XGBoost (OVR)"
else:
    raise RuntimeError("No trained ensemble found. Run the training cells first.")



# pick a random TSV
tsv_files = sorted(glob.glob(os.path.join(FEATURES_DIR, "*.tsv")))

test_path = np.random.choice(tsv_files)
df_test = pd.read_csv(test_path, sep="\t")
name = Path(test_path).stem

# build features exactly like training
NUM_ALL = [
    "s_rsa","s_phi","s_psi","s_a1","s_a2","s_a3","s_a4","s_a5",
    "t_rsa","t_phi","t_psi","t_a1","t_a2","t_a3","t_a4","t_a5",
]
CAT_ALL = ["s_resn","t_resn","s_ss8","t_ss8"]

for c in ["s_3di_letter","t_3di_letter"]:
    if c in df_test.columns:
        CAT_ALL.append(c)

num_cols_test = [c for c in NUM_ALL if c in df_test.columns]
cat_cols_test = [c for c in CAT_ALL if c in df_test.columns]

Xt = df_test[num_cols_test + cat_cols_test].copy()
for c in num_cols_test:
    Xt[c] = pd.to_numeric(Xt[c], errors="coerce").fillna(Xt[c].mean())
Xt = pd.get_dummies(Xt, columns=cat_cols_test, dummy_na=False)

# align to training columns
Xt = Xt.reindex(columns=model_columns, fill_value=0).astype(np.float32)

# probabilities and predictions
P = np.column_stack([ensemble[c].predict_proba(Xt)[:, 1] for c in classes]).astype(np.float32)
P = pd.DataFrame(P, columns=classes, index=Xt.index)
yhat = P.idxmax(axis=1)

print(f"Random test file: {name}")
print(f"model: {model_name} \n")

# evaluate if labels exist

mask = df_test["Interaction"].notna()
y_true = df_test.loc[mask, "Interaction"].astype(str)
y_pred = yhat.loc[mask]

acc = accuracy_score(y_true, y_pred)
ba  = balanced_accuracy_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)


print("Accuracy:", f"{acc:.4f}")
print("Balanced Accuracy:", f"{ba:.4f}")
print("MCC:", f"{mcc:.4f}")
print("Average Precision (macro):", "n/a" if np.isnan(apm) else f"{apm:.4f}")
print("ROC AUC (macro):", "n/a" if np.isnan(aucm) else f"{aucm:.4f}")




Random test file: 6fbz
model: XGBoost (OVR) 

Accuracy: 0.5517
Balanced Accuracy: 0.7771
MCC: 0.3009
Average Precision (macro): 0.3630
ROC AUC (macro): 0.8708


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
